#### 작물클래스
> Common wheat, Maize, Sugar beet

> 나머지는 잡초

In [ ]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
import cv2
import albumentations as A
from albumentations import ToTensorV2
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset,DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from dataclasses import dataclass,field

In [ ]:
rows = []

main_path=Path("/kaggle/input/v2-plant-seedlings-dataset")
output_path=r"/kaggle/working/"
for class_dir in main_path.iterdir():
    if class_dir.is_dir():
        label=class_dir.name
        for img_path in class_dir.glob("*"):
            if img_path.suffix.lower() == ".png":
                rows.append({"path":str(img_path),"label":label})

df=pd.DataFrame(rows)
df.tail()



In [ ]:
classes = sorted(df["label"].unique())
mapping = {c:i for i,c in enumerate(classes)}
df["targets"]=df['label'].map(mapping)


In [ ]:
train_df,tmp_df=train_test_split(df,test_size=0.3,stratify=df["targets"])
val_df,test_df=train_test_split(tmp_df,test_size=0.3,stratify=tmp_df["targets"])

print(train_df.shape)
print(val_df.shape)
print(test_df.shape)

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import math

def visualize_samples(df, n_samples=24, ncols=6):
    """
    df: path, targets 컬럼을 가진 DataFrame
    n_samples: 시각화할 이미지 개수
    ncols: 한 줄에 보여줄 이미지 수
    """
    df_sample = df.sample(n=min(n_samples, len(df)), random_state=12)

    nrows = math.ceil(len(df_sample) / ncols)
    fig, axes = plt.subplots(nrows, ncols, figsize=(ncols * 3, nrows * 3))
    axes = axes.flatten()

    for ax, (_, row) in zip(axes, df_sample.iterrows()):
        img = Image.open(row["path"]).convert("RGB")
        w, h = img.size
        target = row["targets"]

        ax.imshow(img)
        ax.set_title(f"target: {target}\n{h} x {w}", fontsize=10)
        ax.axis("off")

    # 남는 subplot 제거
    for ax in axes[len(df_sample):]:
        ax.axis("off")

    plt.tight_layout()
    plt.show()

visualize_samples(train_df, n_samples=30, ncols=6)


In [ ]:
tr_augmentor=A.Compose([
    A.Resize(224,224,p=1),
    A.HorizontalFlip(p=0.1),
    A.VerticalFlip(p=0.1),
    A.HueSaturationValue(
    hue_shift_limit=5,     # 🔴 매우 중요: 녹색계열에서 Hue를 세게 주면 재앙적 결과초래. 녹색 → “녹색 범위 안에서만” 흔들림
    sat_shift_limit=20,
    val_shift_limit=15,
    p=0.5),
    A.Normalize(mean=(0.485, 0.456, 0.406),std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])
val_augmentor=A.Compose([
    A.Resize(224,224,p=1),
    A.Normalize(mean=(0.485, 0.456, 0.406),std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

In [ ]:
class PlantCustom(torch.utils.data.Dataset):
    def __init__(self,path,targets,augmentor=None):
        self.path=path
        self.targets=targets
        self.augmentor=augmentor
    def __len__(self):
        return len(self.path)
    def __getitem__(self,idx):
        raw_path=self.path[idx]
        raw_img=cv2.cvtColor(cv2.imread(raw_path),cv2.COLOR_BGR2RGB)
        if self.augmentor is not None:
            img=self.augmentor(image=raw_img)['image']
        else:
            raise FileError("augment logic must be need")
        targets=torch.tensor(self.targets[idx],dtype=torch.long)
        return img,targets

In [ ]:
train_custom=PlantCustom(train_df["path"].to_list(), train_df["targets"].to_list(), augmentor=tr_augmentor)
val_custom=PlantCustom(val_df["path"].to_list(),val_df["targets"].to_list(),augmentor=val_augmentor)
test_custom=PlantCustom(test_df["path"].to_list(),test_df["targets"].to_list(),augmentor=val_augmentor)

train_loader=DataLoader(train_custom,batch_size=32,shuffle=True,num_workers=4,pin_memory=True)
val_loader=DataLoader(val_custom,batch_size=32,shuffle=False,num_workers=4,pin_memory=True)
test_loader=DataLoader(test_custom,batch_size=32,shuffle=False,num_workers=4,pin_memory=True)

print("ok")

In [ ]:
k,v=next(iter(train_loader))
print(k.shape)
print(v.shape)

In [ ]:
from torchvision.models import resnet50, ResNet50_Weights
from torchinfo import summary
from torchmetrics import Accuracy
from torchmetrics.classification import MulticlassRecall

model = resnet50(weights=ResNet50_Weights.DEFAULT)
model.fc = nn.Sequential(
    nn.Linear(2048, 300),
    nn.ReLU(inplace=True),
    nn.Dropout(0.2),
    nn.Linear(300, 100),
    nn.ReLU(inplace=True),
    nn.Dropout(0.2),
    nn.Linear(100, 12),
)

device="cuda" if torch.cuda.is_available() else "cpu"
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)
scheduler=ReduceLROnPlateau(optimizer,factor=0.5,patience=3,threshold=0.001)
metric_acc=Accuracy(task="multiclass",num_classes=12)
metric_rec=MulticlassRecall(num_classes=12,average='macro')
loss_func=nn.CrossEntropyLoss()

In [ ]:
# summary(model,input_size=(32,3,224,224),depth=4)

In [ ]:


def is_improvement(best_value,value):
    if value>best_value:
        return True
    else:
        return False
   
class EarlyStop:
    def __init__(self,early_stop,stop_count):
        self.early_stop=early_stop
        self.stop_count=0
    def stop_logic(self,best_value,value):
        if is_improvement(best_value,value):
            self.stop_count=0
            return False
        else:
            if self.stop_count>=self.early_stop:
                print("early_stopped")
                return True
            else:
                self.stop_count+=1
                return False

class Weights_ChkPoint:
    def __init__(self,path):
        self.path=path
        self.count=0
    def save(self,value):
        if is_improvement(best_value,value):
            torch.save(state_dict,f"{self.count}_{value:.4f}.pt")
            self.count+=1
            return False
        else:
            return False

In [ ]:
from typing import List

@dataclass
class History:
    training_accuracy:List[float]=field(default_factory=list)
    training_recall:List[float]=field(default_factory=list)
    training_loss:List[float]=field(default_factory=list)
    val_accuracy:List[float]=field(default_factory=list)
    val_recall:List[float]=field(default_factory=list)
    val_loss:List[float]=field(default_factory=list)
history=History()


class Trainer:
    def __init__(self,train_loader,val_loader,model,optimizer,loss_func,
                 scheduler,metric_acc,metric_rec,device,history,mode="min"):
        self.model=model.to(device)
        self.train_loader=train_loader
        self.val_loader=val_loader
        self.optimizer=optimizer
        self.loss_func=loss_func
        self.scheduler=scheduler
        self.metric_acc=metric_acc.to(device)
        self.metric_rec=metric_rec.to(device)
        self.device=device
        self.history=history
        if mode=="max":
            self.best_value=float('-inf')
        else:
            self.best_value=float('inf')

    def training_epoch(self,epoch):
        self.metric_acc.reset()
        self.metric_rec.reset()
        self.model.train()
        loss_sum=0.0
        avg_loss=0.0
        with tqdm(total=len(self.train_loader),desc=f"training {epoch}",leave=True) as bar:
            for batch_idx,(x_train,y_train) in enumerate(self.train_loader):
                x_train=x_train.to(self.device)
                y_train=y_train.to(self.device)
                logits=self.model(x_train)
                loss=self.loss_func(logits,y_train)
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                loss_sum+=loss.item()
                avg_loss=loss_sum/(batch_idx+1)
                preds=logits.argmax(dim=1)   # dim=-1과 같다. (B,12)  1번 dim 즉 행에 대해서
                self.metric_acc.update(preds, y_train)
                self.metric_rec.update(preds, y_train)
                bar.update(1)

                if batch_idx%10==0:
                    acc=self.metric_acc.compute().item()
                    recall=self.metric_rec.compute().item()
                    bar.set_postfix({"acc": acc, "recall":recall, "loss":avg_loss,"epoch":epoch})
            return self.metric_acc.compute().item(), self.metric_rec.compute().item(),avg_loss  

    def validating_epoch(self,epoch):
        self.metric_acc.reset()
        self.metric_rec.reset()
        self.model.eval()
        loss_sum=0
        avg_loss=0.0
        with tqdm(total=len(self.val_loader),desc=f"validating {epoch}", leave=True) as bar:
            with torch.no_grad():
                for batch_idx,(x_val,y_val) in enumerate(self.val_loader):
                    x_val=x_val.to(self.device)
                    y_val=y_val.to(self.device)
                    logits=self.model(x_val)
                    loss=self.loss_func(logits,y_val)

                    preds=logits.argmax(dim=-1)
                    self.metric_acc.update(preds,y_val)
                    self.metric_rec.update(preds,y_val)
                    loss_sum+=loss.item()
                    avg_loss=loss_sum/(batch_idx+1)
                    bar.update(1)
                    if batch_idx%10==0:
                        acc=self.metric_acc.compute().item()
                        recall=self.metric_rec.compute().item()
                        bar.set_postfix({"acc": acc, "recall":recall, "loss":avg_loss,"epoch":epoch})
                return self.metric_acc.compute().item(), self.metric_rec.compute().item(),avg_loss

    
    def fit(self,epochs,early_stop,path):
        stop_count=0   
        for epoch in range(epochs):
            training_accuracy,training_recall,training_loss=self.training_epoch(epoch)
            self.history.training_accuracy.append(training_accuracy)
            self.history.training_recall.append(training_recall)
            self.history.training_loss.append(training_loss)
            val_accuracy,val_recall,val_loss=self.validating_epoch(epoch)
            self.history.val_accuracy.append(val_accuracy)
            self.history.val_recall.append(val_recall)
            self.history.val_loss.append(val_loss)
            self.scheduler.step(val_loss)

            if self.best_value>val_loss:
                self.best_value=val_loss
                stop_count=0
                torch.save(self.model.state_dict(),os.path.join(path,f"{epoch}_{val_loss}.pt"))
            else:
                stop_count+=1
                if stop_count>=early_stop:
                    print(f"early_stopped. current epoch : {epoch}")
                    return self.history
                    
        return self.history

In [ ]:
t=Trainer(train_loader,val_loader,model,optimizer,loss_func,scheduler,metric_acc,metric_rec,device,history,mode="min")
history=t.fit(30,10,output_path)

In [ ]:
model.load_state_dict(torch.load("/kaggle/input/resnet50-v2plant-seedling-dataset-weights/23_0.14800940760029038.pt"))
print('done')

In [ ]:
!ls /kaggle/input

In [ ]:
from sklearn.metrics import confusion_matrix

class Predictor:
    def __init__(self, model, test_loader, device):
        self.model = model.to(device)
        self.test_loader = test_loader
        self.device = device

    def pred(self):
        pred_list, true_list = [], []
        self.model.eval()
        with tqdm(total=len(self.test_loader), desc="predicting", leave=True) as bar:
            with torch.no_grad():
                for x, y in self.test_loader:
                    x = x.to(self.device)
                    y = y.to(self.device)
                    logits = self.model(x)
                    preds = logits.argmax(dim=-1)

                    pred_list.extend(preds.detach().cpu().tolist())
                    true_list.extend(y.detach().cpu().tolist())
                    bar.update(1)

        return pred_list, true_list


In [ ]:
p=Predictor(model,test_loader,device)
pred,true=p.pred()

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

cm = confusion_matrix(true, pred)
acc = accuracy_score(true, pred)
print(acc, cm.shape)

In [ ]:
test_df['preds']=pred   # 당연히 df의 행의 갯수와 리스트의 원소의 수가 같아야 성립
test_df.head(20)

In [ ]:
print(mapping)
preds_map={}
for k,v in mapping.items():
    preds_map[v]=k

test_df['preds_label']=test_df['preds'].map(preds_map)
test_df.head()

In [ ]:
wrong_df=test_df[test_df["targets"]!=test_df["preds"]]


In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

def show_wrong_predictions_overlay(
    df,
    max_images=16,
    cols=4,
    hspace=0.45,   # ⬅️ 행 간격
    wspace=0.15    # ⬅️ 열 간격
):
    # 1️⃣ 오답만 필터링
    df = df[df["targets"] != df["preds"]].head(max_images)

    rows = (len(df) + cols - 1) // cols
    fig = plt.figure(figsize=(cols * 4, rows * 4))

    for idx, (_, row) in enumerate(df.iterrows()):
        img = Image.open(row["path"]).convert("RGB")

        ax = fig.add_subplot(rows, cols, idx + 1)
        ax.imshow(img)
        ax.axis("off")

        # 상단: 정답 라벨
        ax.text(
            0.5, 1.05,
            f"GT : {row['label']}",
            transform=ax.transAxes,
            ha="center", va="bottom",
            fontsize=11,
            color="green",
            fontweight="bold"
        )

        # 하단: 예측 라벨
        ax.text(
            0.5, -0.12,
            f"PRED : {row['preds_label']}",
            transform=ax.transAxes,
            ha="center", va="top",
            fontsize=10,
            color="red",
            fontweight="bold"
        )

    # 2️⃣ subplot 간격 직접 제어 (핵심)
    plt.subplots_adjust(hspace=hspace, wspace=wspace)
    plt.show()


show_wrong_predictions_overlay(
    test_df,
    max_images=12,
    cols=5,
    hspace=0.8,   # 행 간격 더 넓게
    wspace=0.2
)


## 위 결과만 보면 Loose Silky-bent와 Black-grass간 혼동이 제일 크다는 것을 알수 있다. 